# 05: Architecture Inference & Scaffolding

KindScript has two productivity commands:
- **`ksc infer`** — scans your project, detects the architectural pattern, and generates `architecture.ts`
- **`ksc scaffold`** — reads your architecture definitions and creates the directory structure

Together they form a workflow:
1. Have an existing project? → `infer` detects and generates definitions
2. Starting fresh? → Write definitions, then `scaffold` creates the structure
3. Either way → `check` enforces the rules

## Setup

In [ ]:
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; output: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  const output = (new TextDecoder().decode(stdout) + new TextDecoder().decode(stderr)).trim();
  if (output) console.log(output);
  console.log(`\nExit code: ${code}`);
  return { code, output };
}

async function tree(dir: string): Promise<void> {
  const cmd = new Deno.Command("find", {
    args: [dir, "-type", "f"],
    stdout: "piped",
  });
  const { stdout } = await cmd.output();
  const files = new TextDecoder().decode(stdout).trim().split("\n")
    .map(f => f.replace(dir + "/", ""))
    .sort();
  console.log(files.join("\n"));
}

console.log("Setup complete.");

---

## Part 1: Inferring Architecture from Existing Code

You have a TypeScript project with `src/domain/`, `src/application/`, and `src/infrastructure/` directories. KindScript can detect the pattern and generate everything for you.

In [ ]:
// Create a project with an existing structure
const DEMO = Deno.makeTempDirSync({ prefix: "ksc-infer-" });
Deno.mkdirSync(`${DEMO}/src/domain`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/application`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/infrastructure`, { recursive: true });

Deno.writeTextFileSync(`${DEMO}/src/domain/entity.ts`, `
export interface Order {
  id: string;
  items: string[];
  total: number;
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/application/handler.ts`, `
import { Order } from '../domain/entity';

export function processOrder(order: Order): void {
  console.log('Processing order:', order.id);
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/infrastructure/repository.ts`, `
import { Order } from '../domain/entity';

const store: Order[] = [];
export function save(order: Order): void { store.push(order); }
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
  include: ["src/**/*.ts"],
}, null, 2));

console.log("Project structure:");
await tree(DEMO);

### Dry run: see what `ksc infer` would generate

In [ ]:
await ksc("infer", DEMO);

KindScript detected **Clean Architecture** and generated:
1. **Kind definitions** — `CleanArchitectureContext`, `DomainLayer`, `ApplicationLayer`, `InfrastructureLayer`
2. **Instance mapping** — maps each kind to its `src/` subdirectory
3. **Contracts** — `noDependency` rules between layers + `purity` for all layers

The `purity` contract on all layers is conservative — you can edit it after generation to only apply to `domain`.

### Write mode: generate the files

In [ ]:
await ksc("infer", "--write", DEMO);

In [ ]:
// See the generated files
console.log("=== Generated architecture.ts ===");
console.log(Deno.readTextFileSync(`${DEMO}/architecture.ts`));

console.log("=== Generated kindscript.json ===");
console.log(Deno.readTextFileSync(`${DEMO}/kindscript.json`));

In [ ]:
// Verify: ksc check should pass on the inferred architecture
console.log("=== Running ksc check on the inferred architecture ===");
await ksc("check", DEMO);

Deno.removeSync(DEMO, { recursive: true });

The inferred architecture validates immediately — `ksc infer` generates contracts that match your current code.

---

## Part 2: Scaffolding a New Project

Starting a new project? Write your `architecture.ts` first, then let `ksc scaffold` create the directory structure.

In [ ]:
// Create a project with just config and architecture definitions — no src/ yet
const SCAFFOLD = Deno.makeTempDirSync({ prefix: "ksc-scaffold-" });

Deno.writeTextFileSync(`${SCAFFOLD}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true },
  include: ["src/**/*.ts", "architecture.ts"],
}, null, 2));

Deno.writeTextFileSync(`${SCAFFOLD}/kindscript.json`, JSON.stringify({
  definitions: ["architecture.ts"],
}, null, 2));

Deno.writeTextFileSync(`${SCAFFOLD}/architecture.ts`, `
interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}

interface ContractConfig {
  noDependency?: [string, string][];
  purity?: string[];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}

export interface CleanContext extends Kind<"CleanContext"> {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface ApplicationLayer extends Kind<"ApplicationLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app: CleanContext = {
  kind: "CleanContext",
  location: "src",
  domain: { kind: "DomainLayer", location: "src/domain" },
  application: { kind: "ApplicationLayer", location: "src/application" },
  infrastructure: { kind: "InfrastructureLayer", location: "src/infrastructure" },
};

export const contracts = defineContracts<CleanContext>({
  noDependency: [
    ["domain", "infrastructure"],
    ["domain", "application"],
  ],
  purity: ["domain"],
});
`.trimStart());

console.log("Project defined (no src/ directory yet):");
await tree(SCAFFOLD);

### Dry run: see what `ksc scaffold` would create

In [ ]:
await ksc("scaffold", SCAFFOLD);

The scaffold plan shows:
- Directories for each layer (`src/domain/`, `src/application/`, `src/infrastructure/`)
- Stub `index.ts` files in each directory

### Write mode: create the directories

In [ ]:
await ksc("scaffold", "--write", SCAFFOLD);

In [ ]:
// See the created structure
console.log("\nProject structure after scaffold:");
await tree(SCAFFOLD);

// See what a stub file looks like
console.log("\n=== src/domain/index.ts ===");
console.log(Deno.readTextFileSync(`${SCAFFOLD}/src/domain/index.ts`));

In [ ]:
// Verify: ksc check passes on the freshly scaffolded project
console.log("=== Running ksc check on scaffolded project ===");
await ksc("check", SCAFFOLD);

Deno.removeSync(SCAFFOLD, { recursive: true });

The scaffolded project passes all contracts immediately. You can now start implementing your domain logic inside the pre-created directories, knowing that architectural rules are enforced from day one.

### Scaffold is non-destructive

If you run scaffold on a project that already has some directories, it **skips** existing ones and only creates what's missing. Safe to run repeatedly.

---

## Summary

| Command | Direction | Use case |
|---------|-----------|----------|
| `ksc infer` | Code → Definitions | Existing project, generate architecture.ts |
| `ksc scaffold` | Definitions → Code | New project, create directory structure |
| `ksc check` | Validate | Ongoing, enforce contracts |

**Typical workflows:**
- **Existing project**: `infer --write` → edit contracts → `check`
- **New project**: write `architecture.ts` → `scaffold --write` → implement → `check`
- **CI pipeline**: `check` on every PR